## Extract Data from Warframe API

To avoid unnecessary calls to the API, first check if the csv are already saved:

In [6]:
import os
import pandas as pd

files = os.listdir('warframe_api_data')

get_file = []
if 'warframes.csv' not in files:
    # includes archwings and necramechs
    get_file.append('warframes')

if 'weapons.csv' not in files:
    # includes arch-melees/guns and zaw components (no kitguns)
    get_file.append('weapons')

Request data if needed:

In [7]:
for file in get_file:
    import requests

    # API call
    url = f'https://api.warframestat.us/{file}'
    response = requests.get(url=url)
    json_data = response.json()

    # Keep only item name, type and category
    data = pd.DataFrame(
        [
            {'name': item['name'], 'type': item['type'], 'category': item['category'],
             'isPrime': item['isPrime']}
            for item in json_data
        ]
    )

    # Save as csv using pandas
    data.to_csv(f'warframe_api_data/{file}.csv')

Join data into a single data frame:

In [19]:
# Combine dataframes
data = pd.concat(
    objs=[pd.read_csv('warframe_api_data/warframes.csv'),
    pd.read_csv('warframe_api_data/weapons.csv')]
).\
    reset_index().\
    loc[:, ['name', 'type', 'category', 'isPrime']]

# Preallocate columns for future data 
data['obtained'] = False
data['max_rank'] = False

# Show dataframe
data.head()

,name,type,category,isPrime,obtained,max_rank
0,Amesha,Archwing,Archwing,False,False,False
1,Ash,Warframe,Warframes,False,False,False
2,Ash Prime,Warframe,Warframes,True,False,False
3,Atlas,Warframe,Warframes,False,False,False
4,Atlas Prime,Warframe,Warframes,True,False,False


## Find Matching Items

In [20]:
import pickle
import numpy as np

# Load obtained items list from pickle
with open('output/names.pkl', 'rb') as f:
    # Treat names as an array
    name_list_screenshots = np.array(pickle.load(f))

# Remove blank cases
name_list_screenshots = [name for name in name_list_screenshots if name != '']
len(name_list_screenshots)

512

In [21]:
data.loc[1,:]

name              Ash
type         Warframe
category    Warframes
isPrime         False
obtained        False
max_rank        False
Name: 1, dtype: object

In [ ]:
# Copy of name_list_screenshots to avoid unwanted modifications
names = name_list_screenshots.copy()

# Loop over all item names from the official Warframe API
for row_idx in range(data.shape[0]):
    # Split name to check if every word is in screenshot names
    splitted_name = data.loc[row_idx, 'name'].lower().split(' ')

    # Stop early if no more names to check
    if len(names) == 0:
        print('Early Finish')
        break

    # Loop over screenshot names
    for name_idx in range(len(names)):
        screenshot_name = names[name_idx]

        # Check if every word is in screenshot name
        for word in splitted_name:
            if word not in screenshot_name:
                # Skip to next screenshot name, no need to check
                # the rest of the words
                break
        else:
            # If no break happens: match occurred
            # Delete screenshot name from list (non-primes are 
            # always deleted first because of alphabetical order)
            names = np.delete(names, name_idx)

            # Mark item as obtained
            data.loc[row_idx, 'obtained'] = True

            # Check if item is Max Ranked
            if 'max' in screenshot_name:
                # Register as Max Ranked
                data.loc[row_idx, 'max_rank'] = True

            # Check next item
            break

data.to_csv('output/results.csv')
data.head(10)

,name,type,category,isPrime,obtained,max_rank
0,Amesha,Archwing,Archwing,False,False,False
1,Ash,Warframe,Warframes,False,True,True
2,Ash Prime,Warframe,Warframes,True,True,True
3,Atlas,Warframe,Warframes,False,False,False
4,Atlas Prime,Warframe,Warframes,True,False,False
5,Banshee,Warframe,Warframes,False,True,True
6,Banshee Prime,Warframe,Warframes,True,True,True
7,Baruuk,Warframe,Warframes,False,False,False
8,Baruuk Prime,Warframe,Warframes,True,False,False
9,Bonewidow,Warframe,Warframes,False,False,False
